In [5]:
"""
Minimal character-level Vanilla RNN model. Written by Andrej Karpathy (@karpathy)
BSD License
"""
import numpy as np

# data I/O
data = open('shakespeare.txt', 'r').read() # should be simple plain text file
chars = list(set(data))
data_size, vocab_size = len(data), len(chars)
print ('data has %d characters, %d unique.' % (data_size, vocab_size))
char_to_ix = { ch:i for i,ch in enumerate(chars) }
ix_to_char = { i:ch for i,ch in enumerate(chars) }

# hyperparameters
hidden_size = 100 # size of hidden layer of neurons
seq_length = 25 # number of steps to unroll the RNN for
learning_rate = 1e-1

# model parameters
Wxh = np.random.randn(hidden_size, vocab_size)*0.01 # input to hidden
Whh = np.random.randn(hidden_size, hidden_size)*0.01 # hidden to hidden
Why = np.random.randn(vocab_size, hidden_size)*0.01 # hidden to output
bh = np.zeros((hidden_size, 1)) # hidden bias
by = np.zeros((vocab_size, 1)) # output bias

def lossFun(inputs, targets, hprev):
  """
  inputs,targets are both list of integers.
  hprev is Hx1 array of initial hidden state
  returns the loss, gradients on model parameters, and last hidden state
  """
  xs, hs, ys, ps = {}, {}, {}, {}
  hs[-1] = np.copy(hprev)
  loss = 0
  # forward pass
  for t in range(len(inputs)):
    xs[t] = np.zeros((vocab_size,1)) # encode in 1-of-k representation
    xs[t][inputs[t]] = 1
    hs[t] = np.tanh(np.dot(Wxh, xs[t]) + np.dot(Whh, hs[t-1]) + bh) # hidden state
    ys[t] = np.dot(Why, hs[t]) + by # unnormalized log probabilities for next chars
    ps[t] = np.exp(ys[t]) / np.sum(np.exp(ys[t])) # probabilities for next chars
    loss += -np.log(ps[t][targets[t],0]) # softmax (cross-entropy loss)
  # backward pass: compute gradients going backwards
  dWxh, dWhh, dWhy = np.zeros_like(Wxh), np.zeros_like(Whh), np.zeros_like(Why)
  dbh, dby = np.zeros_like(bh), np.zeros_like(by)
  dhnext = np.zeros_like(hs[0])
  for t in reversed(range(len(inputs))):
    dy = np.copy(ps[t])
    dy[targets[t]] -= 1 # backprop into y. see http://cs231n.github.io/neural-networks-case-study/#grad if confused here
    dWhy += np.dot(dy, hs[t].T)
    dby += dy
    dh = np.dot(Why.T, dy) + dhnext # backprop into h
    dhraw = (1 - hs[t] * hs[t]) * dh # backprop through tanh nonlinearity
    dbh += dhraw
    dWxh += np.dot(dhraw, xs[t].T)
    dWhh += np.dot(dhraw, hs[t-1].T)
    dhnext = np.dot(Whh.T, dhraw)
  for dparam in [dWxh, dWhh, dWhy, dbh, dby]:
    np.clip(dparam, -5, 5, out=dparam) # clip to mitigate exploding gradients
  return loss, dWxh, dWhh, dWhy, dbh, dby, hs[len(inputs)-1]

def sample(h, seed_ix, n):
  """ 
  sample a sequence of integers from the model 
  h is memory state, seed_ix is seed letter for first time step
  """
  x = np.zeros((vocab_size, 1))
  x[seed_ix] = 1
  ixes = []
  for t in range(n):
    h = np.tanh(np.dot(Wxh, x) + np.dot(Whh, h) + bh)
    y = np.dot(Why, h) + by
    p = np.exp(y) / np.sum(np.exp(y))
    ix = np.random.choice(range(vocab_size), p=p.ravel())
    x = np.zeros((vocab_size, 1))
    x[ix] = 1
    ixes.append(ix)
  return ixes

n, p = 0, 0
mWxh, mWhh, mWhy = np.zeros_like(Wxh), np.zeros_like(Whh), np.zeros_like(Why)
mbh, mby = np.zeros_like(bh), np.zeros_like(by) # memory variables for Adagrad
smooth_loss = -np.log(1.0/vocab_size)*seq_length # loss at iteration 0
while True:
  # prepare inputs (we're sweeping from left to right in steps seq_length long)
  if p+seq_length+1 >= len(data) or n == 0: 
    hprev = np.zeros((hidden_size,1)) # reset RNN memory
    p = 0 # go from start of data
  inputs = [char_to_ix[ch] for ch in data[p:p+seq_length]]
  targets = [char_to_ix[ch] for ch in data[p+1:p+seq_length+1]]

  # sample from the model now and then
  if n % 100 == 0:
    sample_ix = sample(hprev, inputs[0], 200)
    txt = ''.join(ix_to_char[ix] for ix in sample_ix)
    print ('----\n %s \n----' % (txt, ))

  # forward seq_length characters through the net and fetch gradient
  loss, dWxh, dWhh, dWhy, dbh, dby, hprev = lossFun(inputs, targets, hprev)
  smooth_loss = smooth_loss * 0.999 + loss * 0.001
  if n % 100 == 0: print ('iter %d, loss: %f' % (n, smooth_loss)) # print progress
  
  # perform parameter update with Adagrad
  for param, dparam, mem in zip([Wxh, Whh, Why, bh, by], 
                                [dWxh, dWhh, dWhy, dbh, dby], 
                                [mWxh, mWhh, mWhy, mbh, mby]):
    mem += dparam * dparam
    param += -learning_rate * dparam / np.sqrt(mem + 1e-8) # adagrad update

  p += seq_length # move data pointer
  n += 1 # iteration counter 

data has 99992 characters, 62 unique.
----
 WaNIvKcqXRAv;z;Vn

MZtzq.PeGiq'XpDMQaJOnm!'DpfxIfeTWX:yZXL-Ad;UCBt?QlYXEnp dLemdVGkKwFe'z-gxVSkXXX VVNLqe-MioFUYLic-ypmHEtv,zOZUGKXF;mTRxeU.GXL-lZNCqQmjt?o,aTuC?:W,E;uRymV!BIjU,Q!t
rAkaDJqW
hoCNrd?xJO 
----
iter 0, loss: 103.178356
----
 
h:i ib.tkS f 
y tertdrafh hettfnt n h ceuidfhoibSnbr:heodsnawTtTtsTsmaic'AnsiiSo A
UT:Stuie:k hnoygsesvi aRo nTd
E'ada :S Sd 
oiOrtsct  i eoaoi.eaa gS
b  R y,land ufrA eko t aiiatd A tduraeeLd AuHldS 
----
iter 100, loss: 102.873292
----
   mll TacLsfk.oohd do altmo Ity oe,eblcad  adi ,Tae a akv ht lo lta
nyo  teole 
g adotaeaoea  n laX es ayano mlthoohooesAto a asose da aeBspy iy r ,ao a ee   a
a
ryoklc ldtts h
be ncTui kaoTmoOtf  omi 
----
iter 200, loss: 101.613952
----
 onpoftItir
roeonuIEwoeYur,hegempGE,dt s 
elksy ,
rw npt
ikOstH wuoo,elDs,.:'DoSwft o 
 d ya
e  s
Ut o :rdo,y aoo  ! d ,o, r  ieesUds mttmocatrniAerg wm hrr.!tOitOs teCyffa CgenpthSL ot,Gslri aoeaT f o 
----
iter 300, loss: 100.275063
----

----
 ingally hy owe uauric ileche, Loee
Bite lor, yne thlo kile, fuve of ildous fpa gont worgun a:
no de the be or felas opin, cof yse
Astell leas holas, I nula palhraim weo waot Puve, aut coot htouly yofd 
----
iter 3500, loss: 62.553908
----
 nytor; mass thor tor,
Theif go ald wat rit mfreendirltor touly neorgo so-Toadiy uth
HaKs rathare forge range gfut and ae pis, and Woad hifk nhe and at be d rouns bop Ind:
Th hoth: if sier thisgheicwor 
----
iter 3600, loss: 62.148252
----
 d thincet sun d,I


ARAnd that ghllir the Thon hoder il sulveic, therfaseas, buthherni'd cras 'a, foun toug' coonreres woblyed'icse ith if toir,

or the, I ay hered.

LAFALARP:
Hos  arrs,
And aaon
Nat 
----
iter 3700, loss: 61.898758
----
 nds fr.
me thit my tfebon ey thllh theplat sad and had tred padeer haop I
Whert os whs fatldy? wusm rot welg the tlladidt an
Bith meve, ball to wow gell iare

FEURI:
Ser Ould tow and and a por spom th 
----
iter 3800, loss: 61.581186
----
 maary rounI if,-

LA:
TOLHTEA:
Asr

----
 A:
Herowchoutl tr my bear duy them herlmce dho ttainduwiogn hevind nonctadg fime youd aresnt theariundeser bir st;
Whace my'lersevapet nod ard thuth axy, wisstou host iw sope:
Wyou,
Orsfour on'd and p 
----
iter 7000, loss: 56.292270
----
 tser beas, nounien?

COLIRT PBON:
A thar ere endiend live whreW,
NuN me you tutig, lored mcat, my the lmoo pramastly jeaa .
Shurerren ke pantagrill,
Terlr fot I wagin.

SEKEW:
Thor.
Hill nther mestttr 
----
iter 7100, loss: 56.219616
----
 nd of viine dot dy tarins shwele to thad ist to inret if dous: Rith'ls il
Wes to domeilds shiin en's ite sird fot sing thes uge s'thenos YoveeguX ie svilm, wous;
At thill wake not of al, gathes dur mu 
----
iter 7200, loss: 56.169639
----
 d, your hur my labe fosare; I
Buldethe hith ass
is ure in gere
hight, in lien mis thelu it t,owe latkes araine sichpare muke and and sakn ding aed, thesamancut, an gore ard arll artit I meswiet vavise 
----
iter 7300, loss: 56.036402
----
 irn huss bemem dot ougluctf to sto

----
 live,
Brtnyousteshe at dake le may muthir lote gos, onk cy OI diy;
Sy the ether grop hase,
Yedo, thin on,
I'll hin ther thave o pro him yod my heve got in will ftod fiviwe thet cameselop d'lrets thajo 
----
iter 10500, loss: 54.262360
----
 ug to to tole
Maveve to ?is in doad sund whe eer and, kayta; osed fere daswfene fake alesn, and lord deaxap
Lot mokt thomet iffow mene wor heve you.
And in wiled,
Bet ute pot enats ipol be's aen gute  
----
iter 10600, loss: 54.260049
----
 e ar ur poon shalln fanorms sounds pass Bringt ably has im; be cord wals cow hase theives nithids lepir bes the has peaversur goodar silosets lom theis:
!oves antos whet, you lor gare nuvera

DS VERIL 
----
iter 10700, loss: 54.144041
----
 o ill:
Sithye in boren: saciis throf pory lices if ler; annnomtsis und noory the gevim! ionp wille to thall thell ye?

LLARS:
Mast not besers to fellowinl but to frichtay ! fame meske ay you keeprang  
----
iter 10800, loss: 54.046670
----
 e furtind
Bem.

YANG:
Namst I 

----
  mes fonday, wee saingrarcdast,

TIO:
Sool same my thair sake whuw, you I wee not: whou the wow ostaclir sritheeld!

ATEAS:
Brere med ome'n teaint
ban,
And, beast hink in hoon anker ant arbutmey
Do to 
----
iter 13900, loss: 52.596929
----
 l thebound sudtekinct monlle nood you hae deatise hing mune evere hid theed.

ANNNLLRON:
Coudid or.i gogare and thes shate exlle, thalg bo'd.

VOLENTUK:
why cof ip lavecun of vall ar we mas trus I lon 
----
iter 14000, loss: 52.609230
----
 st
Do be wou thise
An heard he eqdan,
Ines, ar hofe the eh frach bens frost has: of me thell ore I whe whervers uf me fhoodemnd be her brepend mad, ksprensse mirsile now, I le hears's hit noirt, wer;  
----
iter 14100, loss: 52.740576
----
 st ane.

ULEANSIC AND:
I oprurernvere,
Wome berce thour frish unter, whe.

ICKOLEL:
Yuld, sat, am mime, , your, lile sare bo toou.

Arks intarsey the mark,
I for as dim suteln undeb theer, our my, Bem 
----
iter 14200, loss: 52.783579
----
 in thorlinbes
Fiththerink the 

...

----
 are its live
From the wheerouthin forzen of this
Tralt on datfecior.
Goo, will I will not heaks, chave the base brace
To beess,
Nolamdare? Ho lave factink me whrke that, biouny now,
And do feeans hand 
----
iter 268900, loss: 43.444687
----
 no be bmeen,
The, us.

CALDAES:
Wish to a thind and rive yougn,
So I deak consent.

SCEQUCE:
Lirth with deak to buse to no shad onsenaore.

QUEEN MARESA:
Thy I the gruble I frown bord?

Fecond Cray an 
----
iter 269000, loss: 43.445409
----
 
So pucpoin been:
Hers their to parn.

MARNIAs:
Pod brie, that apor ofter her are life ange exprety of a kingur, her ramy intuny your sope it be neres nooperand,
Sim expung or reay?

TRUDUS: Of's by m 
----
iter 269100, loss: 43.245374
----
 s:
IF oven the Grot if like, I heved in hit retcchy
They I, my deppuns if the Tarte, and is abest jolling we poplings ull take disedfe their bron o'd thee.
Be in atly you have who amphat my ladbant,
W 
----
iter 269200, loss: 43.154047
----
 ntargitast bedirer I lopt 

----
 t bet uffextner:
Of me I would, you may have in the ciners, and neaktirs go rint five.

Second Appruct, shith hop of ones uftused to eved moepunn: full forders.

POLUCBOF:
Shet the how.

AJULY:
With t 
----
iter 272300, loss: 43.397354
----
 preve' gonds to slinger,
To suft the ningr?

SIRGNO:
Hance, the eaver?

CHEELIO:
You
Lfy, as lowh more me hear as twucker heabe?

ARDELANCER:
Nets hould proard to sisther sort if the prape mener ceans 
----
iter 272400, loss: 43.460268
----
 , thee poot where louds I wall of is gragior casmang ford seedrughters this meing a foolmon Lose's ourse: hat derey as wrreckess as a morns with in my;
But are a pavens as are with noiss them shas tho 
----
iter 272500, loss: 43.356211
----
 ade budtactte you lame our thousty sees a chams up are no my it mak unod shay teve,
Out bears, that I wall peegaield in of my lindod
where move,
Hawe,
Upbot eses beed, birsth
thece lenteblas gainstoar 
----
iter 272600, loss: 43.406037
----
  wrucheme go, wanr couraly

----
 :
Now sowe the give in manior.'d not solamaw: I a courty,
O, enfuveed such doactad.
Why shard, yob borrge;
Held tooghiles moung! suther, the maret of this my say to hair fellot:
And,
Is so thize kisly 
----
iter 275700, loss: 43.661912
----
 itlm't
Who prave Cairs, a man this staw wea minst all noth jord to them that you mage boagutin bray to endir,
With youl with he love seember!

DUKE CFALSIL:
Tho chound I shavime fore, contol's a good  
----
iter 275800, loss: 43.710243
----
 old of than
incusedles strengh Breasp thee,
Thall cithe, frabretant bellisand me goad sid doyorl'd my waugh at in sid ban, your sinare.

PRIMAND:
Curet aingrep and hus.
Vay! But your lostned harmast n 
----
iter 275900, loss: 43.657041
----
 all pruve your gose, sinsiry
that over me to love the paghed we love!

CAGHIO:
Stround, to it, and arband he fordh bath as greveress'd and must,
Wher suld lood
Fore mace stupo,
Sotly, Co,
If thee swer 
----
iter 276000, loss: 43.597363
----
 ire, the poing thunce, dec

----
 rue:
To kasty Joldent, within thow, wherter, tell vans, Cay mordert with our hand!
Ale wyour not of in stuccely
shall thine with my gons for undousine leve:
Thou now, it noth,
Some
Count sweeds,
By th 
----
iter 279100, loss: 45.683358
----
 derslast nome, and now quee and all yet ford you letheray's, gilst with: not is heve, and le heaving follust for is will Englay at hear of to eveeld lighean;-speage
Magrece you worur,
Theign's Eash
Th 
----
iter 279200, loss: 45.613816
----
 we to stiry with thoughticed.

QUESNINA:
Hat,
What know, I'll brouty onerat a recly a nampars:
To of blems, abosp not thuslgeting him blome eneths anded hesp in thoublome thee moje onfostly with thy d 
----
iter 279300, loss: 45.714362
----
 es entace
This ligt.
Owes.

ATHEST:
Thou host in her,
Hose.

First, and men,
To lout
how a deaven it sweeve teectart, fling thear ever.

COCAUDUS:
Thoct have everamser to conjat: of the, heave,
come w 
----
iter 279400, loss: 45.734240
----
 do-y
Beluses of he wapp we

KeyboardInterrupt: 